# Download data

In [ ]:
# Code to download file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


In [ ]:
# lab 10
testid = "1laS3maLWQNNuTTOT-QRxn0fBOaqCMWlk"
trainid = "19Od31ooLtxt78tUIgMjUgeIojsAZbkiD"
valid = "1tiWCzJKMTUJf0k_QzoLHErb1VI0IjKC5"

downloaded = drive.CreateFile({'id':testid})
downloaded.GetContentFile('test_data.csv')
downloaded = drive.CreateFile({'id':trainid})
downloaded.GetContentFile('train_data.csv')
downloaded = drive.CreateFile({'id':valid})
downloaded.GetContentFile('val_data.csv')

In [ ]:
# lab 10
# import dataset
import pandas as pd

df_train = pd.read_csv('train_data.csv')
df_test = pd.read_csv('test_data.csv')
df_val = pd.read_csv('val_data.csv')
df_train.sample(10)

,sents,labels
415,On Monday 19 December the UN Security Council ...,O B-Temporal I-Temporal I-Temporal B-Organisat...
533,Our position remains as strong as ever – we wi...,O O O O O O O O B-Organisation O O O O O O O O...
226,A Foreign Office spokesman said :,B-Person I-Person I-Person I-Person O O
265,It was completed in 1956 by the German company...,O O O O B-Temporal O O B-Nationality O B-Organ...
328,Over the last year the Global Coalition has la...,O B-Temporal I-Temporal I-Temporal B-Organisat...
332,"For now though , Omar says the library is help...",O O O O B-Person O B-Location I-Location O O O...
257,""" We mourn the loss of one of our own , Navy S...",O B-Organisation O O O O O O O O O B-Person I-...
73,- Ensure an effective posture .,O O O O O O
343,It ’ s essential that the regime and its backe...,O O O O O B-Organisation I-Organisation O O O ...
259,The victims included his nephews working in th...,B-Organisation I-Organisation O B-Organisation...


In [ ]:
df_train.sents[0]

'Operation Steel Curtain ( Arabic : ا ل ح ج ا ب ا ل ف و ل ا ذ ي Al Hejab Elfulathi ) was a military operation executed by coalition forces in early November 2005 to reduce the flow of foreign insurgents crossing the border and joining the Iraqi insurgency .'

In [ ]:
def get_tokens(dataframe):

  sentences = list(dataframe.sents)

  tokenized_sentences = []
  for sentence in sentences:
    tokens = sentence.split(" ")
    tokens_lower = [s.lower() for s in tokens]
    tokenized_sentences.append(tokens_lower)

  return tokenized_sentences

In [ ]:
def get_labels(dataframe):

  labels = list(dataframe.labels)

  target_y = []
  for label in labels:
    target_y.append(label.split(" "))

  return target_y

In [ ]:
train_data = get_tokens(df_train)
target_y_train = get_labels(df_train) 

In [ ]:
validation_data = get_tokens(df_val)
target_y_validation = get_labels(df_val) 

In [ ]:
test_data = get_tokens(df_test)
# target_y_test = get_labels(df_test) 

In [ ]:
train_data[0]

In [ ]:
target_y_train[0]

# Preprocess data

Generate word_to_ix and tag_to_ix

In [ ]:
# lab 9
word_to_ix = {}
for sentence in train_data+validation_data+test_data:
    for word in sentence:
        word = word.lower()
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
word_list = list(word_to_ix.keys())

START_TAG = "<START>"
STOP_TAG = "<STOP>"
tag_to_ix = {START_TAG:0, STOP_TAG:1}
for tags in target_y_train+target_y_validation:
    for tag in tags:
        if tag not in tag_to_ix:
            tag_to_ix[tag] = len(tag_to_ix)


POS

In [ ]:
tokens = train_data+validation_data+test_data

In [ ]:
# https://blog.csdn.net/JasonJarvan/article/details/79955664?ops_request_misc=&request_id=&biz_id=102&utm_term=python%20pos%20tag%E7%94%9F%E6%88%90&utm_medium=distribute.pc_search_result.none-task-blog-2~all~sobaiduweb~default-0-.pc_search_result_control_group&spm=1018.2226.3001.4187
import nltk
nltk.download('averaged_perceptron_tagger')

tags_pos = []
for scentence in tokens:
  pos_tags =nltk.pos_tag(scentence)
  ret = []
  for word,pos in pos_tags:
    ret.append(pos)
  tags_pos.append(ret)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [ ]:
tags_pos

dependency path

In [ ]:
# lab7
import spacy
#PrettyTable is a Python library for generating simple ASCII tables.
from prettytable import PrettyTable

nlp = spacy.load("en_core_web_sm")

dependency_path = []

for sentence in tokens:
  # covert list to string
  
  parse = nlp(''.join(sentence))
  temp = []
  for token in parse:
    temp.append(token.dep_)
  dependency_path.append(temp)


In [ ]:
dependency_path

Embedding pos tag and dependency path

In [ ]:
from gensim.models import FastText

word_emb_model_pos = FastText(sentences = tags_pos, size=25, word_ngrams=5, window=5, min_count=1, workers=2, sg=1)

In [ ]:
embedding_pos={}
for i in range(len(tokens)):
    for j in range(len(tokens[i])):
        embedding_pos[tokens[i][j]] = word_emb_model_pos[tags_pos[i][j]]



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  after removing the cwd from sys.path.


In [ ]:
embedding_pos

In [ ]:
from gensim.models import FastText

word_emb_model_dp = FastText(sentences = dependency_path, size=25, word_ngrams=5, window=5, min_count=1, workers=2, sg=1)

In [ ]:
embedding_dp={}
for i in range(len(tokens)):
    for j in range(len(tokens[i])):
      try:
        embedding_dp[tokens[i][j]] = word_emb_model_dp[dependency_path[i][j]]
      except:
        continue



/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """


In [ ]:
embedding_dp

# Baseline

In [ ]:
# baseline
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-25") 

EMBEDDING_DIM = 25

embedding_matrix_base = []
for word in word_list:
    try:
        embedding_matrix_base.append(word_emb_model.wv[word])
    except:
        embedding_matrix_base.append([0]*EMBEDDING_DIM)
embedding_matrix_base = np.array(embedding_matrix_base)
embedding_matrix_base.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:10: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  # Remove the CWD from sys.path while we load stuff.


(3957, 25)

convert dataset into idxs

In [ ]:
# lab 9
def to_index(data, to_ix):
    input_index_list = []
    for sent in data:
        input_index_list.append([to_ix[w] for w in sent])
    return input_index_list

train_input_index =  to_index(train_data,word_to_ix)
train_output_index = to_index(target_y_train,tag_to_ix)
val_input_index = to_index(validation_data,word_to_ix)
val_output_index = to_index(target_y_validation,tag_to_ix)
test_input_index = to_index(test_data,word_to_ix)
# test_output_index = to_index(target_y_test,tag_to_ix)

In [ ]:
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF_base(nn.Module):

    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim):
        super(BiLSTM_CRF_base, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)

        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix_base))
        
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=1, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2).to(device),
                torch.randn(2, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [ ]:
import numpy as np
def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    return predicted, ground_truth, accuracy

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 50

model = BiLSTM_CRF_base(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM).to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, weight_decay=1e-4)

In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime

for epoch in range(10):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section

Epoch:1, Training loss: 15010.89, train acc: 0.7463, val loss: 4735.04, val acc: 0.6953, time: 84.81s
Epoch:2, Training loss: 9081.21, train acc: 0.7930, val loss: 3678.69, val acc: 0.7404, time: 84.88s
Epoch:3, Training loss: 6968.59, train acc: 0.8134, val loss: 3255.39, val acc: 0.7569, time: 84.73s
Epoch:4, Training loss: 5871.36, train acc: 0.8269, val loss: 2957.17, val acc: 0.7719, time: 84.76s
Epoch:5, Training loss: 5119.91, train acc: 0.8409, val loss: 2702.64, val acc: 0.7802, time: 85.01s
Epoch:6, Training loss: 4519.54, train acc: 0.8538, val loss: 2546.33, val acc: 0.7846, time: 84.76s
Epoch:7, Training loss: 4069.54, train acc: 0.8626, val loss: 2392.62, val acc: 0.7911, time: 84.65s
Epoch:8, Training loss: 3687.21, train acc: 0.8754, val loss: 2255.28, val acc: 0.7947, time: 85.40s
Epoch:9, Training loss: 3357.28, train acc: 0.8834, val loss: 2229.22, val acc: 0.7973, time: 84.82s
Epoch:10, Training loss: 3062.14, train acc: 0.8954, val loss: 2139.57, val acc: 0.8013, t

In [ ]:
torch.save(model, "ass2_1.pt")

In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, _ = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))
# The log below is the sample output for this section

In [ ]:
from sklearn.metrics import f1_score

f1_sequence = []
f1_score_num_1 = f1_score(y_true_decode,y_pred_decode,average='weighted')
f1_sequence.append(f1_score_num_1)

In [ ]:
f1_score_num_1

0.7814615945746894

In [ ]:
f1_sequence

[0.7814615945746894]

# NER model

Generate Embedding Matrix

In [ ]:
#lab 9
import numpy as np
import gensim.downloader as api
word_emb_model = api.load("glove-twitter-100") 

EMBEDDING_DIM = 150

embedding_matrix = []
for word in word_list:
    word_embedding_token = []
    try:
        word_embedding_token.extend(word_emb_model.wv[word])
        word_embedding_token.extend(embedding_pos[word])
        word_embedding_token.extend(embedding_dp[word])
        embedding_matrix.append(word_embedding_token)
    except:
        embedding_matrix.append([0]*EMBEDDING_DIM)
embedding_matrix = np.array(embedding_matrix)
embedding_matrix.shape

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  if sys.path[0] == '':


(3957, 150)

In [ ]:
embedding_matrix[0]

convert dataset into idxs

In [ ]:
def cosine_similarity(vector1, vector2):
  dot_product = 0.0
  normA = 0.0
  normB = 0.0
  for a, b in zip(vector1, vector2):
    dot_product += a * b
    normA += a ** 2
    normB += b ** 2
  if normA == 0.0 or normB == 0.0:
    return 0
  else:
    return round(dot_product / ((normA**0.5)*(normB**0.5)) * 100, 2)


Model

In [ ]:
# lab 9
import math
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

class BiLSTM_CRF(nn.Module):



    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, layers, attention_method = None):
        super(BiLSTM_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)


        # stacked layer
        self.layers = layers
        # attention method
        self.attention_method = attention_method
       


        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        # find the optimal number of stacked layers
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def cal_attention(self, hidden, encoder_hiddens, attention_method):
        if method == "Dot Product":
            # bmm: https://pytorch.org/docs/master/generated/torch.bmm.html
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        elif method == "Scale Dot Product":
            # COMPLETE THIS PART - Scale Dot Product calculation method
            attn_weights = F.softmax(1/np.sqrt(self.hidden_size)*torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        elif method == "Content-base":
            # COMPLETE THIS PART - Scale Dot Product calculation method
            attn_weights = F.softmax(cosine_similarity(hidden, encoder_hiddens.unsqueeze(0)),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        return concat_output

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)


        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_lstm_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq


In [ ]:
# lab 9
import math
import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.optim as optim

torch.manual_seed(1)

def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


# Compute log sum exp in a numerically stable way for the forward algorithm
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))

# Bi-LSTM without CRF
class BiLSTM_without_CRF(nn.Module):



    def __init__(self, vocab_size, tag_to_ix, embedding_dim, hidden_dim, layers, attention_method = None):
        super(BiLSTM_without_CRF, self).__init__()
        self.embedding_dim = embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.tag_to_ix = tag_to_ix
        self.tagset_size = len(tag_to_ix)


        # stacked layer
        self.layers = layers
        # attention method
        self.attention_method = attention_method



        self.word_embeds = nn.Embedding(vocab_size, embedding_dim)

        """Here we use the embedding matrix as the initial weights of nn.Embedding"""
        self.word_embeds.weight.data.copy_(torch.from_numpy(embedding_matrix))
        
        # find the optimal number of stacked layers
        self.lstm = nn.LSTM(embedding_dim, hidden_dim // 2,
                            num_layers=self.layers, bidirectional=True)

        # Maps the output of the LSTM into tag space.
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size)

        # Matrix of transition parameters.  Entry i,j is the score of
        # transitioning *to* i *from* j.
        self.transitions = nn.Parameter(
            torch.randn(self.tagset_size, self.tagset_size))

        # These two statements enforce the constraint that we never transfer
        # to the start tag and we never transfer from the stop tag
        self.transitions.data[tag_to_ix[START_TAG], :] = -10000
        self.transitions.data[:, tag_to_ix[STOP_TAG]] = -10000

        self.hidden = self.init_hidden()

    def init_hidden(self):
        return (torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device),
                torch.randn(2*self.layers, 1, self.hidden_dim // 2).to(device))

    def _forward_alg(self, feats):
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size), -10000.).to(device)
        # START_TAG has all of the score.
        init_alphas[0][self.tag_to_ix[START_TAG]] = 0.

        # Wrap in a variable so that we will get automatic backprop
        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(
                    1, -1).expand(1, self.tagset_size)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        alpha = log_sum_exp(terminal_var)
        return alpha

    def cal_attention(self, hidden, encoder_hiddens, attention_method):
        if method == "Dot Product":
            # bmm: https://pytorch.org/docs/master/generated/torch.bmm.html
            attn_weights = F.softmax(torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        elif method == "Scale Dot Product":
            # Scale Dot Product calculation method
            attn_weights = F.softmax(1/np.sqrt(self.hidden_size)*torch.bmm(hidden, encoder_hiddens.T.unsqueeze(0)),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        elif method == "Content-base":
            # Content-base calculation method
            attn_weights = F.softmax(cosine_similarity(hidden, encoder_hiddens.unsqueeze(0)),dim=-1)
            attn_output = torch.bmm(attn_weights, encoder_hiddens.unsqueeze(0))
            concat_output = torch.cat((attn_output[0], hidden[0]), 1)

        return concat_output

    def _get_lstm_features(self, sentence):
        self.hidden = self.init_hidden()
        embeds = self.word_embeds(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(embeds, self.hidden)
        lstm_out = lstm_out.view(len(sentence), self.hidden_dim)
        lstm_feats = self.hidden2tag(lstm_out)
        return lstm_feats

    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).to(device)
        tags = torch.cat([torch.tensor([self.tag_to_ix[START_TAG]], dtype=torch.long).to(device), tags])
        for i, feat in enumerate(feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        score = score + self.transitions[self.tag_to_ix[STOP_TAG], tags[-1]]
        return score

    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, self.tagset_size), -10000.).to(device)
        init_vvars[0][self.tag_to_ix[START_TAG]] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size):
                # next_tag_var[i] holds the viterbi variable for tag i at the
                # previous step, plus the score of transitioning
                # from tag i to next_tag.
                # We don't include the emission scores here because the max
                # does not depend on them (we add them in below)
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            # Now add in the emission scores, and assign forward_var to the set
            # of viterbi variables we just computed
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        # Transition to STOP_TAG
        terminal_var = forward_var + self.transitions[self.tag_to_ix[STOP_TAG]]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        # Follow the back pointers to decode the best path.
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == self.tag_to_ix[START_TAG]  # Sanity check
        best_path.reverse()
        return path_score, best_path

    def neg_log_likelihood(self, sentence, tags):
        feats = self._get_lstm_features(sentence)
        forward_score = self._forward_alg(feats)
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        
        lstm_feats = self._get_lstm_features(sentence)
        return lstm_feats, torch.argmax(lstm_feats, -1)
        # lstm_feats, tags = model(sentence_in)
        # loss = loss_func(lstm_feats, targets)

Function for accuracy

In [ ]:
import numpy as np
def cal_acc(model, input_index, output_index):
    ground_truth = []
    predicted = []
    for i,idxs in enumerate(input_index):
        ground_truth += output_index[i]
        score, pred = model(torch.tensor(idxs, dtype=torch.long).to(device))
        predicted += pred
    accuracy = sum(np.array(ground_truth) == np.array(predicted))/len(ground_truth)
    return predicted, ground_truth, accuracy

# Testing

# Model 2 - attention - Dot product, stacked layer - 1, crf - with crf

Initialize the model

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 150


# attention - Dot Product, stacked layer - 1, crf - use
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, layers = 1, 
                   attention_method = "Dot Product").to(device)

optimizer = optim.SGD(model.parameters(), lr=0.015, weight_decay=1e-4)

loss_func = nn.CrossEntropyLoss()


Train the model

In [ ]:
train_data=train_input_index+val_input_index
train_label=train_output_index+val_output_index

In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime

for epoch in range(10):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section

Epoch:1, Training loss: 13276.36, train acc: 0.7736, val loss: 4025.32, val acc: 0.7294, time: 87.72s
Epoch:2, Training loss: 7683.94, train acc: 0.8141, val loss: 3180.44, val acc: 0.7613, time: 87.66s
Epoch:3, Training loss: 5848.81, train acc: 0.8436, val loss: 2785.30, val acc: 0.7829, time: 87.72s
Epoch:4, Training loss: 4654.94, train acc: 0.8737, val loss: 2557.29, val acc: 0.7975, time: 87.79s
Epoch:5, Training loss: 3737.52, train acc: 0.8960, val loss: 2534.88, val acc: 0.8024, time: 87.83s
Epoch:6, Training loss: 3053.47, train acc: 0.9100, val loss: 2599.05, val acc: 0.7986, time: 87.88s
Epoch:7, Training loss: 2508.40, train acc: 0.9245, val loss: 2607.92, val acc: 0.7984, time: 88.01s
Epoch:8, Training loss: 2107.10, train acc: 0.9413, val loss: 2599.02, val acc: 0.8034, time: 87.94s
Epoch:9, Training loss: 1798.22, train acc: 0.9477, val loss: 2772.83, val acc: 0.8009, time: 88.50s
Epoch:10, Training loss: 1536.85, train acc: 0.9501, val loss: 2860.31, val acc: 0.7903, t

In [ ]:
torch.save(model, "ass2_2.pt")

Testing

In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, _ = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true_decode,y_pred_decode,digits=4))
# The log below is the sample output for this section

In [ ]:
from sklearn.metrics import f1_score


f1_score_num_2 = f1_score(y_true_decode,y_pred_decode,average='weighted')
f1_sequence.append(f1_score_num_2)

In [ ]:
f1_score_num_2

0.7715082808132664

In [ ]:
f1_sequence

[0.7814615945746894, 0.7715082808132664]

# Model 3 - attention - Dot product, stacked layer - 2, crf - with crf

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 150


# attention - Dot Product, stacked layer - 1, crf - use
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, layers = 2, 
                   attention_method = "Dot Product").to(device)

optimizer = optim.SGD(model.parameters(), lr=0.015, weight_decay=1e-4)

loss_func = nn.CrossEntropyLoss()


In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime

for epoch in range(10):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section

Epoch:1, Training loss: 13726.23, train acc: 0.7419, val loss: 4490.61, val acc: 0.6966, time: 96.71s
Epoch:2, Training loss: 9103.29, train acc: 0.7895, val loss: 3460.41, val acc: 0.7442, time: 96.68s
Epoch:3, Training loss: 6774.53, train acc: 0.8254, val loss: 2937.06, val acc: 0.7702, time: 96.54s
Epoch:4, Training loss: 5386.70, train acc: 0.8563, val loss: 2638.79, val acc: 0.7861, time: 96.75s
Epoch:5, Training loss: 4277.58, train acc: 0.8748, val loss: 2570.23, val acc: 0.7977, time: 96.77s
Epoch:6, Training loss: 3494.65, train acc: 0.8905, val loss: 2588.74, val acc: 0.7990, time: 96.56s
Epoch:7, Training loss: 2956.94, train acc: 0.9054, val loss: 2642.56, val acc: 0.7958, time: 96.64s
Epoch:8, Training loss: 2521.38, train acc: 0.9127, val loss: 2750.31, val acc: 0.7981, time: 96.63s
Epoch:9, Training loss: 2062.29, train acc: 0.9357, val loss: 2810.54, val acc: 0.7988, time: 96.60s
Epoch:10, Training loss: 1763.83, train acc: 0.9337, val loss: 3057.58, val acc: 0.8028, t

In [ ]:
torch.save(model, "ass2_3.pt")

In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, _ = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

In [ ]:
from sklearn.metrics import f1_score


f1_score_num_3 = f1_score(y_true_decode,y_pred_decode,average='weighted')
f1_sequence.append(f1_score_num_3)

In [ ]:
f1_sequence

[0.7814615945746894, 0.7715082808132664, 0.7788268436176835]

# Model 4 - attention - Dot product, stacked layer - 3, crf - with crf

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 150


# attention - Dot Product, stacked layer - 1, crf - use
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, layers = 3, 
                   attention_method = "Dot Product").to(device)

optimizer = optim.SGD(model.parameters(), lr=0.015, weight_decay=1e-4)

loss_func = nn.CrossEntropyLoss()


In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime

for epoch in range(10):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section

Epoch:1, Training loss: 16761.34, train acc: 0.6999, val loss: 5535.29, val acc: 0.6485, time: 105.70s
Epoch:2, Training loss: 12354.61, train acc: 0.7363, val loss: 4388.86, val acc: 0.6907, time: 105.50s
Epoch:3, Training loss: 9507.23, train acc: 0.7721, val loss: 3602.12, val acc: 0.7215, time: 105.52s
Epoch:4, Training loss: 7507.93, train acc: 0.8084, val loss: 3092.28, val acc: 0.7531, time: 105.52s
Epoch:5, Training loss: 5980.63, train acc: 0.8447, val loss: 2814.10, val acc: 0.7719, time: 105.50s
Epoch:6, Training loss: 4869.70, train acc: 0.8617, val loss: 2641.50, val acc: 0.7831, time: 105.55s
Epoch:7, Training loss: 4074.18, train acc: 0.8772, val loss: 2657.44, val acc: 0.7812, time: 105.55s
Epoch:8, Training loss: 3471.77, train acc: 0.8867, val loss: 2705.31, val acc: 0.7884, time: 105.43s
Epoch:9, Training loss: 2967.63, train acc: 0.8978, val loss: 2789.51, val acc: 0.7911, time: 105.45s
Epoch:10, Training loss: 2546.70, train acc: 0.9152, val loss: 2874.91, val acc:

In [ ]:
torch.save(model, "ass2_4.pt")

In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, _ = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

In [ ]:
from sklearn.metrics import f1_score


f1_score_num_4 = f1_score(y_true_decode,y_pred_decode,average='weighted')
f1_sequence.append(f1_score_num_4)

In [ ]:
f1_sequence = []
f1_sequence.append(f1_score_num_1)
f1_sequence.append(f1_score_num_2)
f1_sequence.append(f1_score_num_3)
f1_sequence.append(f1_score_num_4)

In [ ]:
f1_sequence

[0.7814615945746894,
 0.7715082808132664,
 0.7788268436176835,
 0.7632831368650179]

# Model 5 - attention - Scale Dot Product, stacked layer - 1, crf - with crf

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 150


# attention - Dot Product, stacked layer - 1, crf - use
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, layers = 1, 
                   attention_method = "Scale Dot Product").to(device)

optimizer = optim.SGD(model.parameters(), lr=0.015, weight_decay=1e-4)

loss_func = nn.CrossEntropyLoss()


In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime

for epoch in range(10):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section

Epoch:1, Training loss: 14120.08, train acc: 0.7780, val loss: 4158.22, val acc: 0.7296, time: 87.82s
Epoch:2, Training loss: 7950.26, train acc: 0.8177, val loss: 3287.44, val acc: 0.7598, time: 88.08s
Epoch:3, Training loss: 5993.37, train acc: 0.8481, val loss: 2864.88, val acc: 0.7856, time: 87.83s
Epoch:4, Training loss: 4722.36, train acc: 0.8736, val loss: 2694.91, val acc: 0.7971, time: 87.83s
Epoch:5, Training loss: 3772.40, train acc: 0.8929, val loss: 2709.77, val acc: 0.7983, time: 87.83s
Epoch:6, Training loss: 3055.07, train acc: 0.9105, val loss: 2664.33, val acc: 0.7996, time: 87.87s
Epoch:7, Training loss: 2559.94, train acc: 0.9235, val loss: 2833.25, val acc: 0.8011, time: 87.92s
Epoch:8, Training loss: 2180.34, train acc: 0.9328, val loss: 2826.57, val acc: 0.8045, time: 87.96s
Epoch:9, Training loss: 1837.77, train acc: 0.9435, val loss: 2857.68, val acc: 0.8036, time: 87.75s
Epoch:10, Training loss: 1542.28, train acc: 0.9448, val loss: 3089.64, val acc: 0.7981, t

In [ ]:
torch.save(model, "ass2_5.pt")

In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, _ = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

In [ ]:
from sklearn.metrics import f1_score


f1_score_num_5 = f1_score(y_true_decode,y_pred_decode,average='weighted')
f1_sequence.append(f1_score_num_5)

In [ ]:
f1_sequence

[0.7814615945746894,
 0.7715082808132664,
 0.7788268436176835,
 0.7632831368650179,
 0.7795319620211295]

# Model 6 - attention - Content base, stacked layer - 1, crf - with crf

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 150


# attention - Dot Product, stacked layer - 1, crf - use
model = BiLSTM_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, layers = 1, 
                   attention_method = "Content-base").to(device)

optimizer = optim.SGD(model.parameters(), lr=0.015, weight_decay=1e-4)

loss_func = nn.CrossEntropyLoss()


In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime

for epoch in range(10):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        loss = model.neg_log_likelihood(sentence_in, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    val_loss = 0
    for i, idxs in enumerate(val_input_index):
        tags_index = val_output_index[i]
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)
        loss = model.neg_log_likelihood(sentence_in, targets)
        val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section

Epoch:1, Training loss: 14129.65, train acc: 0.7761, val loss: 3999.46, val acc: 0.7262, time: 88.02s
Epoch:2, Training loss: 7871.76, train acc: 0.8173, val loss: 3126.08, val acc: 0.7596, time: 88.07s
Epoch:3, Training loss: 5888.96, train acc: 0.8521, val loss: 2708.05, val acc: 0.7865, time: 88.11s
Epoch:4, Training loss: 4620.42, train acc: 0.8749, val loss: 2551.46, val acc: 0.7892, time: 88.10s
Epoch:5, Training loss: 3685.49, train acc: 0.8992, val loss: 2493.61, val acc: 0.7945, time: 89.09s
Epoch:6, Training loss: 2991.31, train acc: 0.9135, val loss: 2596.91, val acc: 0.7994, time: 88.08s
Epoch:7, Training loss: 2516.97, train acc: 0.9288, val loss: 2567.04, val acc: 0.8034, time: 88.23s
Epoch:8, Training loss: 2093.35, train acc: 0.9380, val loss: 2649.52, val acc: 0.8026, time: 88.11s
Epoch:9, Training loss: 1765.92, train acc: 0.9435, val loss: 2867.71, val acc: 0.8011, time: 88.03s
Epoch:10, Training loss: 1563.12, train acc: 0.9522, val loss: 2823.85, val acc: 0.8024, t

In [ ]:
torch.save(model, "ass2_6.pt")

In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, _ = cal_acc(model,val_input_index,val_output_index)

def decode_output(output_list):
    ix_to_tag = {v:k for k,v in tag_to_ix.items()}
    return [ix_to_tag[output] for output in output_list]

y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred)

In [ ]:
from sklearn.metrics import f1_score


f1_score_num_6 = f1_score(y_true_decode,y_pred_decode,average='weighted')
f1_sequence.append(f1_score_num_6)

In [ ]:
f1_sequence

# Model 7 - attention - Dot Product, stacked layer - 1, crf - without crf

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
HIDDEN_DIM = 150


# attention - Dot Product, stacked layer - 1, crf - use
model = BiLSTM_without_CRF(len(word_to_ix), tag_to_ix, EMBEDDING_DIM, HIDDEN_DIM, layers = 1, 
                   attention_method = "Dot Product").to(device)

optimizer = optim.SGD(model.parameters(), lr=0.015, weight_decay=1e-4)

loss_func = nn.CrossEntropyLoss()

In [ ]:
"""Each epoch will take about 1-2 minutes"""

import datetime

for epoch in range(10):  
    time1 = datetime.datetime.now()
    train_loss = 0

    model.train()
    for i, idxs in enumerate(train_input_index):
        tags_index = train_output_index[i]

        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is,
        # turn them into Tensors of word indices.
        sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
        targets = torch.tensor(tags_index, dtype=torch.long).to(device)

        # Step 3. Run our forward pass.
        lstm_feats, tags = model(sentence_in)
        loss = loss_func(lstm_feats, targets)

        # Step 4. Compute the loss, gradients, and update the parameters by
        # calling optimizer.step()
        loss.backward()
        optimizer.step()

        train_loss+=loss.item()

    model.eval()
    # Call the cal_acc functions you implemented as required
    _, _, train_acc = cal_acc(model,train_input_index,train_output_index)
    _, _, val_acc = cal_acc(model,val_input_index,val_output_index)

    # val_loss = 0
    # for i, idxs in enumerate(val_input_index):
    #     tags_index = val_output_index[i]
    #     sentence_in = torch.tensor(idxs, dtype=torch.long).to(device)
    #     targets = torch.tensor(tags_index, dtype=torch.long).to(device)
    #     loss = model.neg_log_likelihood(sentence_in, targets)
    #     val_loss+=loss.item()
    time2 = datetime.datetime.now()

    print("Epoch:%d, Training loss: %.2f, train acc: %.4f, val loss: %.2f, val acc: %.4f, time: %.2fs" %(epoch+1, train_loss,train_acc, val_loss, val_acc, (time2-time1).total_seconds()))

# The log below is the sample output for this section

Epoch:1, Training loss: 854.24, train acc: 0.6999, val loss: 3036.80, val acc: 0.6485, time: 9.22s
Epoch:2, Training loss: 716.03, train acc: 0.6998, val loss: 3036.80, val acc: 0.6485, time: 9.25s
Epoch:3, Training loss: 679.89, train acc: 0.7017, val loss: 3036.80, val acc: 0.6511, time: 9.30s
Epoch:4, Training loss: 655.68, train acc: 0.7130, val loss: 3036.80, val acc: 0.6608, time: 9.27s
Epoch:5, Training loss: 635.98, train acc: 0.7178, val loss: 3036.80, val acc: 0.6653, time: 9.29s
Epoch:6, Training loss: 615.39, train acc: 0.7207, val loss: 3036.80, val acc: 0.6705, time: 9.30s
Epoch:7, Training loss: 596.40, train acc: 0.7247, val loss: 3036.80, val acc: 0.6773, time: 9.33s
Epoch:8, Training loss: 581.22, train acc: 0.7299, val loss: 3036.80, val acc: 0.6830, time: 9.28s
Epoch:9, Training loss: 565.51, train acc: 0.7360, val loss: 3036.80, val acc: 0.6898, time: 9.31s
Epoch:10, Training loss: 545.96, train acc: 0.7416, val loss: 3036.80, val acc: 0.6928, time: 9.26s


In [ ]:
torch.save(model, "ass2_7.pt")

In [ ]:
# Call the cal_acc functions you implemented as required
y_pred, y_true, _ = cal_acc(model,val_input_index,val_output_index)



In [ ]:
y_pred_1 = []
for item in y_pred:
  # tensor(2).item()
  item = item.item()
  y_pred_1.append(item)

In [ ]:
y_pred_1

In [ ]:
ix_to_tag = {idx : tag for tag, idx in tag_to_ix.items()}

In [ ]:
def decode_output(output_list):
    
    return [ix_to_tag[output] for output in output_list]



In [ ]:
y_true_decode = decode_output(y_true)
y_pred_decode = decode_output(y_pred_1)

In [ ]:
from sklearn.metrics import f1_score


f1_score_num_7 = f1_score(y_true_decode,y_pred_decode,average='weighted')
f1_sequence.append(f1_score_num_7)

In [ ]:
f1_sequence

[0.7814615945746894,
 0.7715082808132664,
 0.7788268436176835,
 0.7632831368650179,
 0.7795319620211295,
 0.7891339927713741,
 0.604936071899939]

# Draw the table

Performance Comparison

In [ ]:
from prettytable import PrettyTable
t = PrettyTable(['Model', 'Attention method', 'Stacked layers', 'with/without CRF', 'F1 Score'])
t.add_row(['base model', 'without attention', '1', 'with CRF', f1_sequence[0]])
t.add_row(['Model_2', 'Dot Product', '1', 'with CRF', f1_sequence[1]])
t.add_row(['Model_3', 'Dot Product', '2', 'with CRF', f1_sequence[2]])
t.add_row(['Model_4', 'Dot Product', '3', 'with CRF', f1_sequence[3]])
t.add_row(['Model_5', 'Scaled Dot Product', '1', 'with CRF', f1_sequence[4]])
t.add_row(['Model_6', 'Content Base', '1', 'with CRF', f1_sequence[5]])
t.add_row(['Model_7', 'Dot Product', '1', 'without CRF', f1_sequence[6]])

print(t)

+------------+-------------------+----------------+------------------+--------------------+
|   Model    |  Attention method | Stacked layers | with/without CRF |      F1 Score      |
+------------+-------------------+----------------+------------------+--------------------+
| base model | without attention |       1        |     with CRF     | 0.7814615945746894 |
|  Model_2   |    Dot Product    |       1        |     with CRF     | 0.7715082808132664 |
|  Model_3   |    Dot Product    |       2        |     with CRF     | 0.7788268436176835 |
|  Model_4   |    Dot Product    |       3        |     with CRF     | 0.7632831368650179 |
|  Model_5   | Scale Dot Product |       1        |     with CRF     | 0.7795319620211295 |
|  Model_6   |    Content Base   |       1        |     with CRF     | 0.7891339927713741 |
|  Model_7   |    Dot Product    |       1        |   without CRF    | 0.604936071899939  |
+------------+-------------------+----------------+------------------+----------

Different input embedding

In [ ]:
from prettytable import PrettyTable

t = PrettyTable(['Model', 'F1 Score'])
t.add_row(['base model', f1_sequence[0]])
t.add_row(['base model+pos+dependency prase', f1_sequence[1]])



print(t)

+---------------------------------+--------------------+
|              Model              |      F1 Score      |
+---------------------------------+--------------------+
|            base model           | 0.7814615945746894 |
| base model+pos+dependency prase | 0.7715082808132664 |
+---------------------------------+--------------------+


Different stacked layer

In [ ]:
from prettytable import PrettyTable
t = PrettyTable(['Model', 'Stacked layers', 'F1 Score'])
t.add_row(['base model', '1', f1_sequence[0]])
t.add_row(['Model_2', '1', f1_sequence[1]])
t.add_row(['Model_3', '2', f1_sequence[2]])
t.add_row(['Model_4', '3', f1_sequence[3]])


print(t)

+------------+----------------+--------------------+
|   Model    | Stacked layers |      F1 Score      |
+------------+----------------+--------------------+
| base model |       1        | 0.7814615945746894 |
|  Model_2   |       1        | 0.7715082808132664 |
|  Model_3   |       2        | 0.7788268436176835 |
|  Model_4   |       3        | 0.7632831368650179 |
+------------+----------------+--------------------+


Different attention method

In [ ]:
from prettytable import PrettyTable
t = PrettyTable(['Model', 'Attention method', 'F1 Score'])
t.add_row(['base model', 'without attention', f1_sequence[0]])
t.add_row(['Model_2', 'Dot Product', f1_sequence[1]])
t.add_row(['Model_5', 'Scale Dot Product', f1_sequence[4]])
t.add_row(['Model_6', 'Content Base', f1_sequence[5]])


print(t)

+------------+-------------------+--------------------+
|   Model    |  Attention method |      F1 Score      |
+------------+-------------------+--------------------+
| base model | without attention | 0.7814615945746894 |
|  Model_2   |    Dot Product    | 0.7715082808132664 |
|  Model_5   | Scale Dot Product | 0.7795319620211295 |
|  Model_6   |    Content Base   | 0.7891339927713741 |
+------------+-------------------+--------------------+


with/without CRF

In [ ]:
from prettytable import PrettyTable
t = PrettyTable(['Model', 'with/without CRF', 'F1 Score'])
t.add_row(['base model', 'with CRF', f1_sequence[0]])
t.add_row(['Model_2', 'with CRF', f1_sequence[1]])
t.add_row(['Model_7', 'without CRF', f1_sequence[6]])

print(t)

+------------+------------------+--------------------+
|   Model    | with/without CRF |      F1 Score      |
+------------+------------------+--------------------+
| base model |     with CRF     | 0.7814615945746894 |
|  Model_2   |     with CRF     | 0.7715082808132664 |
|  Model_7   |   without CRF    | 0.604936071899939  |
+------------+------------------+--------------------+
